<a href="https://colab.research.google.com/github/matsonah/ClarkeStatsSpring2022/blob/main/Module_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Module 7 includes stones 45 through 46 and primarily practices outcome 7. For those reading the OER text, we are in [Chapter 8](https://openstax.org/books/introductory-statistics/pages/8-introduction). 

Code Block 1: Prepares Data. Exports "Mulan Sample.csv" for editing in Excel. 

Edit Mulan Sample.csv to include Ethnicity for each observational unit in the sample. Save the file as "Mulan Sample with Ethnicity.csv". 

Code Block 2: Checks that "Mulan Sample with Ethnicity.csv" matches the structure we need. 

Code Block 3: Computes a confidence interval using a formula.

Code Block 4: Bootstraps a fake population for sampling. 

Code Block 5: Bypasses bootstrapping a fake population and draws samples as if a fake population was bootstrapped. 

Code Block 6: Graphs the distribution from bootstrapping. 

Code Block 7: For when you know the proportion and need to build a sample off of that.

Code Block 8: For multiple proportions


In [ ]:
### Code Block 1
# Gather entire cast and crew from 2020 Mulan
Mulan <- read.csv("Mulan Cast and Crew.csv")

Mulan.sample <- 0 

# Generate a sample of 30 people. 
Mulan.sample = data.frame(sort(sample(Mulan$Cast.and.Crew, 30, replace=F)))
Mulan.sample[1,2]="0"
names(Mulan.sample)=c("cnc","ethnicity")
str(Mulan.sample)

# Put it in a CSV for easy manipulation. 
write.csv(Mulan.sample, "Mulan Sample.csv")

# Now open the CSV and add ethnicity data to it. Was the person's ethnicity in alignment with the Mulan story? (yes = 1, no = 0)


In [ ]:
### Code Block 2
Mulan.ethnicity = read.csv("Mulan Sample with Ethnicity.csv")
str(Mulan.ethnicity)

In [ ]:
### Code Block 3
#Mulan.ethnicity = read.csv("Mulan Sample with Ethnicity.csv")
#data = Mulan.ethnicity$yes.no..1.0.

prop = 5/8
total = 80 

data = rep(c(1,0), c(prop*total, (1-prop)*total))

pi_hat= mean(data)   # assumes yes = 1 and no = 0
sample.size = length(data)

stdev=sqrt(pi_hat*(1-pi_hat)/sample.size)
conf.level = 0.95

z.star = qnorm(conf.level) # one possibility
margin.of.error = abs(z.star)*(stdev)

lower.bound = max(pi_hat - margin.of.error,0)
upper.bound = min(pi_hat + margin.of.error,1)

cat(sep="","At a confidence level of ", conf.level*100, "%, the actual proportion of the population is between ", lower.bound," and ", upper.bound, ".")

At a confidence level of 95%, the actual proportion of the population is between 0.5359697 and 0.7140303.

Bootstrap a bigger population.


In [ ]:
### Code Block 4
#data<-0

#Mulan.ethnicity = read.csv("Mulan Sample with Ethnicity.csv")
#data = Mulan.ethnicity$yes.no..1.0.


z_conf=0.95 
alpha = 1-z_conf 
zsc=qnorm(z_conf + alpha/2)


size=length(data)
cat("Sample size:", size, "observational units.\n")

bootdata=cbind(data,data)
i=0
for(i in 1:10){
  bootdata=cbind(bootdata,bootdata)
}

cat("Bootstrapped population size:", length(bootdata), "observational units.\n")

test <- 0
test_runs = 100000
for(i in 1:test_runs){ 
  test[i] = mean(sample(bootdata, size, replace=FALSE))
}

cat(sep="","Bootstrap population paramater is a proportion equal to ", mean(test), ".\n")
cat(sep="","Original sample statistic is a proportion equal to ", mean(data),".\n")

Boot = data.frame(x=test)
Boot_mean = mean(Boot$x)
Boot_sd = sd(Boot$x)
Boot_left=Boot_mean - zsc*Boot_sd
Boot_right=Boot_mean + zsc*Boot_sd 
Boots=c(Boot_left,Boot_right)

cat(sep="", "With a confidence level of ", z_conf*100, "%, the actual population paramater is between ", Boot_left, " and ", Boot_right, ".\n")

Sample size: 1000 observational units.
Bootstrapped population size: 2048000 observational units.
Bootstrap population paramater is a proportion equal to 0.7500037.
Original sample statistic is a proportion equal to 0.75.
With a confidence level of 95%, the actual population paramater is between 0.7232431 and 0.7767643.


Bypass the building of a fake population by drawing a sample with replacement. 

In [ ]:
### Code Block 5

#data<-0

#Mulan.ethnicity = read.csv("Mulan Sample with Ethnicity.csv")
#data = Mulan.ethnicity$yes.no..1.0.


z_conf=0.95 
alpha = 1-z_conf 
zsc=qnorm(z_conf + alpha/2)

size=length(data)
cat("Sample size:", size, "observational units.\n")

test <- 0
test_runs = 100000
for(i in 1:test_runs){
  test[i] = mean( sample(data,size,replace=TRUE))
  }


cat(sep="","Original sample statistic is a proportion equal to ", mean(data),".\n")

Boot = data.frame(x=test)
Boot_mean = mean(Boot$x)
Boot_sd = sd(Boot$x)
Boot_left=Boot_mean - zsc*Boot_sd
Boot_right=Boot_mean + zsc*Boot_sd 
Boots=c(Boot_left,Boot_right)

cat(sep="", "With a confidence level of ", z_conf*100, "%, the actual population paramater is between ", Boot_left, " and ", Boot_right, ".\n")


Sample size: 1000 observational units.
Original sample statistic is a proportion equal to 0.75.
With a confidence level of 95%, the actual population paramater is between 0.7230471 and 0.7768082.


In [ ]:
### Code Block 6   
# Requires running either block 4 or block 5 first.

library(ggplot2)

# Fancy function defined to allow for shading. 
dnorm_sd <- function(x,numsd){
  norm_sd <- dnorm(x,Boot_mean, Boot_sd)
  lb = Boot_mean - numsd*Boot_sd  #left bound 
  rb = Boot_mean + numsd*Boot_sd  #right bound 
  # Force NA values outside interval x in [leftbound, rightbound]:
  norm_sd[x <= lb | x >= rb] <- NA
  return(norm_sd)   #return is the result of the function dnorm_sd 
}

ggplot( Boot, aes(x)) + 
  geom_histogram(aes(y=..density../(2*pi)),binwidth=0.005) + 
  stat_function( fun=dnorm,    args=list(mean=Boot_mean, sd=Boot_sd), col="green", size=2) +
  stat_function( fun=dnorm_sd, args=list(numsd=zsc),  geom="area",  fill="green", alpha=0.3 ) + 
  geom_vline( xintercept=Boots,  linetype="longdash",  col="blue", size=2) 

In [ ]:
### Code Block 7
# Can be run independently of all prior code blocks. 
# For proportions. 


sample_count= 1000
sample_size = 1250
sample_complement = sample_size-sample_count 

data=rep(c(1,0),c(sample_count,sample_complement))   # repeats 1 sample_count times and 0 sample_complement times. 

z_conf=0.997 
alpha = 1-z_conf 
zsc=qnorm(z_conf + alpha/2)

size=length(data)
cat("Sample size:", size, "observational units.\n")

test <- 0
test_runs = 100000
for(i in 1:test_runs){
  test[i] = mean( sample(data,size,replace=TRUE))
  }


cat(sep="","Original sample statistic is a proportion equal to ", mean(data),".\n")

Boot = data.frame(x=test)
Boot_mean = mean(Boot$x)
Boot_sd = sd(Boot$x)
Boot_left=Boot_mean - zsc*Boot_sd
Boot_right=Boot_mean + zsc*Boot_sd 
Boots=c(Boot_left,Boot_right)

cat(sep="", "With a confidence level of ", z_conf*100, "%, the actual population paramater is between ", Boot_left, " and ", Boot_right, ".\n")


In [ ]:
### Code Block 8
# Can be run independently of all prior code blocks. 
# For two proportions. 


sample1_count= 100
sample1_size = 125
sample1_complement = sample1_size-sample1_count 

data1=rep(c(1,0),c(sample1_count,sample1_complement))   # repeats 1 sample_count times and 0 sample_complement times. 

sample2_count= 80
sample2_size = 120
sample2_complement = sample2_size-sample2_count 

data2=rep(c(1,0),c(sample2_count,sample2_complement))   # repeats 1 sample_count times and 0 sample_complement times. 


z_conf=0.997 
alpha = 1-z_conf 
zsc=qnorm(z_conf + alpha/2)

test <- 0
test_runs = 100000
for(i in 1:test_runs){
  test[i] = mean( sample(data1,sample1_size,replace=TRUE)) / mean( sample(data2,sample2_size,replace=TRUE))
  }

Boot = data.frame(x=test)
Boot_mean = mean(Boot$x)
Boot_sd = sd(Boot$x)
Boot_left=Boot_mean - zsc*Boot_sd
Boot_right=Boot_mean + zsc*Boot_sd 
Boots=c(Boot_left,Boot_right)

cat(sep="", "With a confidence level of ", z_conf*100, "%, the actual differenece in population parameters is between ", Boot_left, " and ", Boot_right, ".\n")

With a confidence level of 99.7%, the actual differenece in population parameters is between -0.03279804 and 0.2996421.
